In [1]:
import pandas as pd
import numpy as np
import time
import json
from pprint import pprint

In [2]:
def getTagsDict(df):
    tag_dict = {}
    for index, row in df.iterrows():
        for tag in row['hashtags']:
            if tag in tag_dict:
                tag_dict[tag] = tag_dict[tag] + 1
            else:
                tag_dict[tag] = 1
    sort_dict = sorted(tag_dict.items(), key=lambda kv: kv[1])
    return sort_dict

def get_DataFrame_for_affinity(df,taglist):
    result_df = pd.DataFrame(0, index = np.arange(0,len(df)),columns=taglist)    
    for i in range(len(df)):
        list_of_row = list(df.iloc[i])[0]
        for tag in list_of_row:
            if tag in taglist:
                result_df[tag][i] = 1
    return result_df

In [3]:
## JSON Rawdata 파일 읽어오기
filename = 'allseason_20k_rawdata.json'
allseason = pd.read_json(filename)
base_df = allseason[['id','username','hashtags']]

## id 와 hashtag 만 남기기
base_df.index = base_df['id']
base_df = base_df.drop(columns=['username'])
base_df = base_df.drop(columns=['id'])

## 비어있는 해시태그 리스트 제거 / 중복되는 태그 제거
fashion_df = base_df[base_df.astype(str)['hashtags'] != '[]']
tag_df = list(fashion_df['hashtags'])
for i in range(len(tag_df)):
    tag_df[i] = list(set(tag_df[i]))
fashion_df['hashtags'] = tag_df

## n 개의 고빈도 태그만 사용하도록 결정
n_tags = 1000
fashion_dict = getTagsDict(fashion_df)
fashion_dict_top = fashion_dict[-n_tags:]
fashion_top = []
for i in fashion_dict_top:
    fashion_top.append(i[0])

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [4]:
## 해당 태그로 post-hashtag 매트릭스의 생성
start_time = time.time()
post_hashtag = get_DataFrame_for_affinity(fashion_df,fashion_top)
print("--- %s seconds ---" % (time.time() - start_time))

--- 63.11908936500549 seconds ---


In [7]:
## post corr 연산
start_time = time.time()
hashtag_post = post_hashtag.transpose()
post_corr = hashtag_post.corr()
print("--- %s seconds ---" % (time.time() - start_time))

--- 536.4476792812347 seconds ---


In [8]:
post_corr.head()

,0,1,2,3,4,5,6,7,8,9,...,17634,17635,17636,17637,17638,17639,17640,17641,17642,17643
0,1.000000,0.081518,0.252602,0.444743,0.197828,0.166602,0.178076,0.242189,0.135366,0.361660,...,-0.005952,-0.011116,-0.009597,-0.008748,-0.010877,-0.002243,-0.005952,-0.005025,-0.005025,0.192411
1,0.081518,1.000000,0.102732,0.093579,0.265327,0.015438,0.072423,0.155049,0.115575,0.072423,...,0.221942,0.231557,-0.021231,0.088156,0.237463,-0.004961,-0.013166,0.081518,-0.011116,0.164280
2,0.252602,0.102732,1.000000,0.284021,0.253741,0.097418,0.110360,0.311662,0.173529,0.348204,...,0.211058,0.222745,0.054157,0.061956,0.105623,-0.003487,-0.009253,0.122395,-0.007812,0.312255
3,0.444743,0.093579,0.284021,1.000000,0.223272,0.188761,0.200226,0.272458,0.152860,0.405376,...,-0.005321,-0.009938,-0.008580,-0.007820,-0.009723,-0.002005,-0.005321,-0.004492,-0.004492,0.217277
4,0.197828,0.265327,0.253741,0.223272,1.000000,0.069318,0.084034,0.242682,0.206860,0.178880,...,0.251879,0.121752,0.036251,0.103345,0.076372,-0.004403,-0.011685,0.093981,-0.009865,0.189395


In [19]:
post_id = fashion_df.index.values
ids = pd.DataFrame(post_id)

In [13]:
start_time = time.time()
without_minus = post_corr.applymap(lambda x: 0 if x < 0 else x)
print("--- %s seconds ---" % (time.time() - start_time))

--- 119.0314028263092 seconds ---


In [16]:
start_time = time.time()
sum_mat = without_minus.sum(axis=1)
print("--- %s seconds ---" % (time.time() - start_time))

--- 9.098252534866333 seconds ---


In [17]:
sum_mat.head()

0    1209.968006
1    1732.790501
2    1966.220255
3    1291.342112
4    1558.053579
dtype: float64

In [45]:
corr_score = pd.DataFrame(sum_mat, columns = ['score'])
corr_score['id'] = ids
corr_score.head()

,score,id
0,1209.968006,BtOm-dkHn5v
1,1732.790501,BtQp-IfHkzq
2,1966.220255,BtRdH3UBQ6i
3,1291.342112,BtOpNQpApMI
4,1558.053579,BtRHyjZAem6


In [48]:
corr_score.sort_values(by = ['score'], ascending = False)

,score,id
16423,3131.509325,BtMd1D1HNU5
2978,3107.264357,BtQmjVNhM04
9027,3042.606433,Bs5FZWDBT-V
2873,2969.937663,BtQDGTZlVRo
220,2914.414819,BtR7Ta9h21T
17401,2900.111623,BtOuKMljNPa
13149,2898.085699,Bs4igLAjgam
7638,2894.670923,Bs3rXlxlBNd
7847,2890.711429,BtB64LZBIoS
17338,2858.618591,BtE3KmJBvkU
